In [1]:
# using fusion fine-tuning strategy
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
# from torch.autograd import VariableW
import torch.optim as optim
import numpy as np
import os
import scipy.sparse as sp
from time import time
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,multilabel_confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import KFold
# from gin import Encoder, GAT
from torch_geometric.loader import DataLoader
import os.path as osp
from tqdm import tqdm
from torch_geometric.nn import MessagePassing, global_add_pool, global_sort_pool, GCNConv, GATConv
from torch_geometric.data import Dataset, InMemoryDataset, Data
from torch_geometric.utils import to_dense_adj
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import numpy as np
sys.path.append('../')
sys.path.append('../../')
sys.path.append('./')
from pytorchtools import EarlyStopping
import matplotlib.pyplot as plt
from tqdm import tqdm
from RGCN import res_GCN, MV_MGCL, cal_GCL
from Optimizer import AdaBound, NAdam
import random

In [2]:
seed = 3407  #3701, 3407
# 生成随机数，以便固定后续随机数，方便复现代码
random.seed(seed)
# 没有使用GPU的时候设置的固定生成的随机数
np.random.seed(seed)
# 为CPU设置种子用于生成随机数，以使得结果是确定的
torch.manual_seed(seed)
# torch.cuda.manual_seed()为当前GPU设置随机种子
torch.cuda.manual_seed(seed)

In [3]:
# #模型扰动
# def gen_ran_output(data, model, vice_model, eta):
#     for (adv_name,adv_param), (name,param) in zip(vice_model.named_parameters(), model.named_parameters()):
#         if name.split('.')[0] == 'proj_head': #映射层参数不变化。
#             adv_param.data = param.data
#         else:
#             adv_param.data = param.data + eta * torch.normal(0,torch.ones_like(param.data)*param.data.std()).to(device)   #GNN模型参数扰动，Gasussio       
#     z2 = vice_model(data.x, data.edge_index, data.edge_attr, data.batch, data.num_graphs)
#     return z2

In [4]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)

# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()

In [5]:
#dataset generator
class SG(InMemoryDataset):
    global d_type
    def __init__(self, root, transform=None, pre_transform=None,pre_filter=None):
        super().__init__(root, transform=None, pre_transform=None, pre_filter=None)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_dir(self):
        return osp.join(self.root, '/raw/'+d_type)
    
    @property
    def raw_file_names(self):
        return os.listdir(self.root+'/raw/'+d_type)

    @property
    def processed_file_names(self):
        return [d_type+'.pt']

    def process(self):
        data_list = []
        for raw_path in tqdm(self.raw_paths):
            edges = []
            edge_features = []
            nodes = []
            node_features = []
            with open(self.root+raw_path,'r') as f:
                row = f.readline().strip().split(",")
                row = [int(w) for w in row]
                nodes_num = row[0]
                label = row[1:]
                for line,j in zip(f.readlines(),range(nodes_num)):
                    row = line.strip().split()
                    row ,attr = [int(w) for w in row[1:int(row[0])+1]],np.array([float(w) for w in row[int(row[0])+1:]])
                    if attr is not None:
                        node_features.append(attr)
                    if row is not None:
                        for k in row:
                            edge = [j,k]
                            edges.append(edge)
            for edge in edges:
                edge_features.append((node_features[edge[0]]+node_features[edge[1]])/2)
            num_edges = len(edges)
            new_edges = torch.zeros((2,num_edges),dtype=torch.long)
            for edge,index in zip(edges,range(num_edges)):
                new_edges[0,index] = edge[0]
                new_edges[1,index] = edge[1]
            label = torch.tensor(label, dtype=torch.float)
            node_features = torch.tensor(node_features,dtype=torch.float)
            nodes = torch.tensor(nodes,dtype=torch.long)
            edge_features = torch.tensor(edge_features,dtype=torch.float)
            data = Data(x=node_features,edge_index=new_edges,edge_attr=edge_features,y=label)
            data_list.append(data)
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [6]:
def generatDataset(random_state = 1):
    global d_type
    dataset = SG('/media/bmw_lab/2eb7452f-ae82-4707-82ed-218aabcd7aaf1/bmw_lab/frankda/CL-Pre_train/dataset/fine-tune/')
    dataset = shuffle(dataset, random_state = random_state)
    length = len(dataset)
    k1 = int(0.9 * length)
    test_dataset = dataset[k1:]
    new_dataset = dataset[:k1]
    my_array = np.empty( shape=(len(new_dataset)), dtype = np.dtype(SG))
    for i in range(len(new_dataset)):
        my_array[i] = new_dataset[i]    # SG datasets to np.object to solve unable to get index through index array
    test_dataloader = DataLoader(test_dataset,batch_size = len(test_dataset))
    # test_dataloader = DataLoader(test_dataset, batch_size = )
    return my_array, test_dataloader

In [7]:
# d_type = 'multi-label'
# new_dataset, test_dataloader = generatDataset()
# for data in test_dataloader:
#     print(data[0].y)


# for train_index, valid_index in Kf.split(new_dataset):
#     train_dataset, valid_dataset = new_dataset[train_index], new_dataset[valid_index]
#     train_dataloader = DataLoader(train_dataset, batch_size = 5)
#     for item in train_dataloader:
#         print(item.y.shape)

In [8]:
def evaluate(predict,t_label,thresh):
    pre = (predict >= thresh).float()
#     print(predict,t_label)
    precision = precision_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
    recall = recall_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
    f1 = f1_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
#     fpr,tpr,_ = roc_curve(t_label.data.cpu(),predict.data.cpu(),average='macro')
#     auc = roc_auc_score(t_label.data.cpu(),predict.data.cpu(),average='macro')
    
    # cal overall accuarcy
    acc = ((((pre == t_label).float().sum(dim=1) / 4) == 1).float().sum()) / t_label.shape[0]
    # cal subset accuarcy
    # 1. arithmetic detect rate
    arithmetic = (pre == t_label)[:,0].float().sum() / t_label.shape[0]
    # 2. reentrancy detect rate
    reentrancy = (pre == t_label)[:,1].float().sum() / t_label.shape[0]
    # 3. Uncheked_low_calls detect rate
    uncheked_low_calls = (pre == t_label)[:,3].float().sum() / t_label.shape[0]
    # 4. Timestamp detect rate
    timestamp = (pre == t_label)[:,2].float().sum() / t_label.shape[0]

    return precision, recall, f1, acc.item(), arithmetic.item(), reentrancy.item(), uncheked_low_calls.item(), timestamp.item()

def test_evaluate(predict,t_label,thresh):
    pre = (predict >= thresh).float()
#     print(predict,t_label)
    precision = precision_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
    recall = recall_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
    f1 = f1_score(t_label.data.cpu(),pre.data.cpu(),average='macro', zero_division = 0)
#     fpr,tpr,_ = roc_curve(t_label.data.cpu(),predict.data.cpu(),average='macro')
    auc = roc_auc_score(t_label.data.cpu(),predict.data.cpu(),average='macro')
    
    # cal overall accuarcy
    acc = ((((pre == t_label).float().sum(dim=1) / 4) == 1).float().sum()) / t_label.shape[0]
    # cal subset accuarcy
    # 1. arithmetic detect rate
    arithmetic = (pre == t_label)[:,0].float().sum() / t_label.shape[0]
    # 2. reentrancy detect rate
    reentrancy = (pre == t_label)[:,1].float().sum() / t_label.shape[0]
    # 3. Uncheked_low_calls detect rate
    uncheked_low_calls = (pre == t_label)[:,3].float().sum() / t_label.shape[0]
      # 4. Timestamp detect rate
    timestamp = (pre == t_label)[:,2].float().sum() / t_label.shape[0]

    return precision, recall, f1, acc.item(), arithmetic.item(), reentrancy.item(), uncheked_low_calls.item(), timestamp.item(),auc

# def evaluate(predict,t_label,thresh):
# #     pre = F.softmax(predict).argmax(dim=1)
#     pre = (predict >= thresh).float()
# #     print(predict,t_label)
#     accuracy = accuracy_score(t_label.data.cpu(),pre.data.cpu())
#     precesion = precision_score(t_label.data.cpu(),pre.data.cpu())
#     recall = recall_score(t_label.data.cpu(),pre.data.cpu())
#     f1 = f1_score(t_label.data.cpu(),pre.data.cpu())
#     fpr,tpr,_ = roc_curve(t_label.data.cpu(), predict.data.cpu())
# #     auc = roc_auc_score(t_label.data.cpu(),predict.data.cpu())

#     return precesion, recall, f1, accuracy,tpr, fpr

# def test_evaluate(predict,t_label,thresh):
#     pre = (predict >= thresh).float()
# #     print(predict,t_label)
#     accuracy = accuracy_score(t_label.data.cpu(),pre.data.cpu())
#     precesion = precision_score(t_label.data.cpu(),pre.data.cpu())
#     recall = recall_score(t_label.data.cpu(),pre.data.cpu())
#     f1 = f1_score(t_label.data.cpu(),pre.data.cpu())
#     fpr,tpr,_ = roc_curve(t_label.data.cpu(),predict.data.cpu())
#     auc = roc_auc_score(t_label.data.cpu(),predict.data.cpu())

#     return precesion, recall, f1, accuracy,tpr, fpr,auc

In [9]:
#init
def init():
    input_dim = 100
    hidden_dim = 64
    output_dim = 32
    num_gc_layers = 10
    k = 100
    lr = 0.0001
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = MV_MGCL(input_dim, hidden_dim, num_gc_layers, output_dim, k).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    # optimizer = NAdam(model.parameters(), lr=lr)
    # optimizer = AdaBound(model.parameters(), lr=1e-3, final_lr=0.01)
#     optimizer = torch.optim.SGD(model.parameters(), lr=lr)
#     optimizer = torch.optim.RMSprop(model.parameters(), lr = lr)
    scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0004 , cycle_momentum = False)
#     scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0004)
    Loss_func = nn.BCEWithLogitsLoss()    # This loss combines a Sigmoid layer and the BCELoss in one single class.
#     Loss_func = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping(patience=30, verbose=True)
    return device, model, optimizer, scheduler, Loss_func, early_stopping

In [10]:
# import pandas as pd
# import networkx as nx
# from torch_geometric.explain.explainer
# # you can make use of this function if you want to visualize the augmented graph
# def vis_graph(data):
#     # fig, (ax0, ax1) = plt.subplots(ncols=2, constrained_layout=True)
#     plt.figure(figsize=(16, 4))
    
#     edge_index = data.edge_index.detach().cpu().numpy()
#     df = pd.DataFrame({'from': edge_index[0], 'to': edge_index[1]})
#     G = nx.from_pandas_edgelist(df, 'from', 'to')
    
#     # plt.subplot(1, 2, 1)
#     pos = nx.fruchterman_reingold_layout(G)
    
#     nx.draw(G, with_labels=True, node_size=100, node_color="skyblue", pos=pos)
#     plt.title("Original")
#     # plt.subplot(1, 2, 2)
#     # edge_index = view1.edge_index.detach().cpu().numpy()
#     # df = pd.DataFrame({'from': edge_index[0], 'to': edge_index[1]})
#     # G = nx.from_pandas_edgelist(df, 'from', 'to')
#     # # pos = nx.fruchterman_reingold_layout(G)
#     # nx.draw(G, with_labels=True, node_size=100, node_color="skyblue", pos=pos)
#     # plt.title("View1")
    

In [11]:
# #测试展示图

# device, model, optimizer, scheduler, Loss_func, early_stopping = init()
# d_type = 'multi-label'
# new_dataset, test_dataloader = generatDataset(random_state = 1)


In [12]:
def train(thresh, train_dataloader, device, model, optimizer, scheduler, Loss_func, mode):
    loss_all = 0
    accuracy = 0
    precision = 0
    recall = 0
    F1 = 0
    model.train()
    for data in train_dataloader:
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        adj = to_dense_adj(data.edge_index).squeeze()
        node_degs = torch.sum(adj, dim = 1) + 1
        node_degs = 1/ node_degs
        node_degs = torch.diag(node_degs).to(device)
        x1 = model.finetune(data.x, data.edge_index, data.edge_attr, data.batch, data.num_graphs, adj, node_degs, mode = mode)
        loss_aug = Loss_func(x1, data.y.view(-1, 4))
        loss_aug.backward()
        loss_all += loss_aug.cpu().item()
        optimizer.step()
        scheduler.step()
    print('train_loss: ', loss_all / len(train_dataloader))
            
def validation(thresh, valid_dataloader, device, model, Loss_func, early_stopping, mode):
    model.eval()
    with torch.no_grad():
        for data in valid_dataloader:
            node_num, _ = data.x.size()
            data = data.to(device)
            adj = to_dense_adj(data.edge_index).squeeze()
            node_degs = torch.sum(adj, dim = 1) + 1
            node_degs = 1/ node_degs
            node_degs = torch.diag(node_degs).to(device)
            x1 = model.finetune(data.x, data.edge_index, data.edge_attr, data.batch, data.num_graphs, adj, node_degs, mode = mode)
            loss_aug = Loss_func(x1, data.y.view(-1, 4))
            precesion,recall,f1_score,accuracy,arithmetic,reentrancy, unchecked_low_calls, timestamp = evaluate(torch.sigmoid(x1), data.y.view(-1, 4), thresh)
    return loss_aug.cpu().item()

def test(thresh, test_dataloader, device, model, Loss_func, mode):
    model.eval()
    with torch.no_grad():
        for data in test_dataloader:
            node_num, _ = data.x.size()
            data = data.to(device)
            adj = to_dense_adj(data.edge_index).squeeze()
            node_degs = torch.sum(adj, dim = 1) + 1
            node_degs = 1/ node_degs
            node_degs = torch.diag(node_degs).to(device)
            x1 = model.finetune(data.x, data.edge_index, data.edge_attr, data.batch, data.num_graphs, adj, node_degs, mode = mode)            
            loss_aug = Loss_func(x1, data.y.view(-1, 4))
            precesion,recall,f1_score,accuracy,arithmetic,reentrancy,unchecked_low_calls,timestamp,auc = test_evaluate(torch.sigmoid(x1), data.y.view(-1, 4), thresh)
    return precesion,recall,f1_score, accuracy, arithmetic, reentrancy, unchecked_low_calls, timestamp, auc, loss_aug.cpu().item()



In [13]:
def start_training(step, train_dataloader, valid_dataloader, test_dataloader, mode):
    global d_type
    global history
    device, model, optimizer, scheduler, Loss_func, early_stopping = init()
    #multi2_k100_no_relu_epoch5_rgcn_no_bn_loss2  multi2_k100_no_relu_epoch5_rgcn_no_bn_loss3 multi2_k100_no_relu_batch64_epoch20_rgcn_no_bn_loss1
    model.load_state_dict(torch.load('./saved_models/pre-train-64-10-(multi2_k100_no_relu_epoch100_rgcn_wo_ln_simplified_projhead_MV_MGCL_cal_GCL2).pt'))  #pre-train-gat-32-10-140 arithmetic AUC=0.92 reentrancy AUC = 0.99 timestamp AUC = 0.9680  unchecked AUC = 1.0
    
    # 冻结gcn和gat的前三层
    # for i in range(6):
    #     for param in model.gat.GAT_layers[i].parameters():
    #         param.requires_grad = False
    #     for param in model.rgcn.layers[i].parameters():
    #         param.requires_grad = False

    # for layer in model.gat.Conv1ds:
    #         for param in layer.parameters():
    #             param.requires_grad = False
    # for layer in model.rgcn.Conv1ds:
    #         for param in layer.parameters():
    #             param.requires_grad = False
    
    # add dropout layer in GNN
    # model.gcn.Conv1ds = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=16,kernel_size = model.gcn.hidden_dim*model.gcn.num_layers,stride = model.gcn.hidden_dim*model.gcn.num_layers),
    #                                 nn.MaxPool1d(kernel_size = 2, stride= 2),
    #                                 nn.Conv1d(in_channels=16, out_channels=32,kernel_size = 4,stride = 4))
    # model.gcn.proj_head = nn.Sequential(nn.Linear(model.gcn.embedding_dim, 512), nn.ReLU(inplace=True), nn.LayerNorm(512), nn.Linear(512, 128))
    # model.gcn.proj_head = nn.Sequential(nn.Linear(model.gcn.embedding_dim, 512), nn.ReLU(inplace=True), nn.LayerNorm(512), nn.Linear(512, 128))

    # model.aug_gcn.proj_head = nn.Sequential(nn.Linear(model.gcn.embedding_dim, 512), nn.BatchNorm1d(512), nn.ReLU(inplace=True), nn.Linear(512, 128))
    # model.aug_gat.Conv1ds = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=16,kernel_size = model.aug_gat.hidden_dim*model.aug_gat.num_layers,stride = model.aug_gat.hidden_dim*model.aug_gat.num_layers),
    #                                 nn.MaxPool1d(kernel_size = 2, stride= 2),
    #                                 nn.Conv1d(in_channels=16, out_channels=32,kernel_size = 4,stride = 4))
    # model.aug_gat.proj_head = nn.Sequential(nn.Linear(model.aug_gat.embedding_dim, 512), nn.ReLU(inplace=True), nn.LayerNorm(512), nn.Linear(512, 128))
    
    # model.nega_gat.proj_head = nn.Sequential(nn.Linear(model.aug_gat.embedding_dim, 512), nn.ReLU(inplace=True), nn.LayerNorm(512), nn.Linear(512, 128))
    # model.proj_head = nn.Sequential(nn.Linear(256, 100), nn.ReLU(inplace=True), nn.Dropout(p=0.4), nn.Linear(100, 64), nn.ReLU(inplace=True), nn.Dropout(p=0.4), nn.Linear(64, 4))
    # concat input 256
    model.proj_head = nn.Sequential(nn.Linear(256, 64), nn.ReLU(inplace=True), nn.LayerNorm(64),  nn.Dropout(p=0.5), nn.Linear(64, 4))
    # model.proj_head = nn.Sequential(nn.Linear(256, 128), nn.ReLU(inplace=True), nn.LayerNorm(128),  nn.Dropout(p=0.4),
    #                                 nn.Linear(128, 64), nn.ReLU(inplace=True), nn.LayerNorm(64),  nn.Dropout(p=0.4), 
    #                                 nn.Linear(64, 4))    
    # only gcn or gat
    # model.proj_head = nn.Sequential(nn.Linear(128, 64), nn.ReLU(inplace=True), nn.LayerNorm(64),  nn.Dropout(p=0.4), nn.Linear(64, 4))   #attempt 1
    # model.proj_head = nn.Sequential(nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(inplace=True), nn.Dropout(p=0.4), nn.Linear(64, 4))                      #attempt 2
    model.to(device)
    for i in range(500):
        train(0.5, train_dataloader, device, model, optimizer, scheduler, Loss_func, mode)  
        v_l = validation(0.5, valid_dataloader, device, model, Loss_func, early_stopping, mode)
#         print('Epoch {}, Loss {}, Acc {}, Precision {}, recall {}, F1-score {}'.format(i, v_l, v_acc, v_pre, v_rec, v_f1))
        early_stopping(v_l, model)
        if early_stopping.early_stop:
            model.load_state_dict(torch.load('checkpoint.pt'))
            break;

    # print('finding the best threshold----')
    # for thresh in np.linspace(0, 1, 100):
    #     precision, recall, f1_score, accuracy, arithmetic, reentrancy, unchecked, timestamp, auc, loss = test(thresh, test_dataloader, device, model, Loss_func)
    #     with open('./simgrace/'+'finetune_9fold_'+d_type+'.txt','a') as f:
    #         print('Thresh %.4f Loss %.4f Acc %.4f arithmetic %.4f reentrancy %.4f timestamp %.4f unchecked %.4f Precision %.4f Recall %.4f F1-score %.4f AUC %.4f' % (thresh, loss, accuracy, arithmetic, reentrancy, timestamp, unchecked, precision, recall, f1_score, auc), file=f)
    # precision, recall, f1_score, accuracy, arithmetic, reentrancy, unchecked, timestamp, auc, loss = test(0.4545, test_dataloader, device, model, Loss_func, mode)
    precision, recall, f1_score, accuracy, arithmetic, reentrancy, unchecked, timestamp, auc, loss = test(0.45, test_dataloader, device, model, Loss_func, mode)
    history['accuracy'].append(accuracy)
    history['loss'].append(loss)
    history['auc'].append(auc)
    history['precision'].append(precision)
    history['recall'].append(recall)
    history['f1'].append(f1_score)
    history['arithmetic'].append(arithmetic)
    history['reentrancy'].append(reentrancy)
    history['timestamp'].append(timestamp)
    history['unchecked'].append(unchecked)
    print('Loss %.4f Acc %.4f arithmetic %.4f reentrancy %.4f timestamp %.4f unchecked %.4f Precision %.4f Recall %.4f F1-score %.4f AUC %.4f' % (loss, accuracy, arithmetic, reentrancy, timestamp, unchecked, precision, recall, f1_score, auc))
    with open('./results/'+'finetune_9fold'+d_type+'.txt','a') as f:
        print('step %2d Loss %.4f Acc %.4f arithmetic %.4f reentrancy %.4f timestamp %.4f unchecked %.4f Precision %.4f Recall %.4f F1-score %.4f AUC %.4f' % (step, loss, accuracy, arithmetic, reentrancy, timestamp, unchecked, precision, recall, f1_score, auc), file=f)

In [14]:
d_type = 'arithmetic'
# detect_types = ['arithmetic','reentrancy','timestamp','unchecked']
detect_types = ['multi-label']

history_all = {'loss':[], 'auc':[], 'precision':[], 'recall':[], 'f1':[], 'accuracy':[], 'arithmetic':[], 'reentrancy':[], 'timestamp':[], 'unchecked':[]}
#concat gcn gat gcn+gat gcn+Ngat Pgat+Ngat Ngat
mode = 'concat'  

for item in detect_types:
    for rand in range(5):
        seed = 3407  #3701, 3407
        # 生成随机数，以便固定后续随机数，方便复现代码
        random.seed(seed)
        # 没有使用GPU的时候设置的固定生成的随机数
        np.random.seed(seed)
        # 为CPU设置种子用于生成随机数，以使得结果是确定的
        torch.manual_seed(seed)
        # torch.cuda.manual_seed()为当前GPU设置随机种子
        torch.cuda.manual_seed(seed)
        history = {'loss':[], 'auc':[], 'precision':[], 'recall':[], 'f1':[], 'accuracy':[], 'arithmetic':[], 'reentrancy':[], 'timestamp':[], 'unchecked':[]}
        d_type = item
        Kf = KFold(n_splits = 9)
        new_dataset, test_dataloader = generatDataset(random_state = rand+1)
        with open('./results/'+'finetune_9fold'+d_type+'.txt','a') as f:
            print(d_type,' start training!', 'random_state=', rand+1, file = f)
        step = 0
        for train_index, valid_index in Kf.split(new_dataset):
            train_dataset, valid_dataset = new_dataset[train_index], new_dataset[valid_index]
            train_dataloader = DataLoader(train_dataset, batch_size = 5, shuffle=True)
            valid_dataloader = DataLoader(valid_dataset, batch_size = len(valid_dataset))
            start_training(step, train_dataloader, valid_dataloader, test_dataloader, mode)
            step += 1
        with open('./results/'+'finetune_9fold'+d_type+'.txt','a') as f:
            print('ave_Loss %.4f, ave_Acc %.4f, ave_arithmetic %.4f, ave_reentrancy %.4f, ave_timestamp %.4f, ave_unchecked %.4f, ave_Precision %.4f, ave_Recall %.4f, ave_F1-score %.4f, ave_AUC %.4f' % (np.array(history['loss']).mean(), np.array(history['accuracy']).mean(), np.array(history['arithmetic']).mean(), np.array(history['reentrancy']).mean(), np.array(history['timestamp']).mean(), np.array(history['unchecked']).mean(), np.array(history['precision']).mean(), np.array(history['recall']).mean(), np.array(history['f1']).mean(), np.array(history['auc']).mean()), file=f)
            print('var_Loss %.4f, var_Acc %.4f, var_arithmetic %.4f, var_reentrancy %.4f, var_timestamp %.4f, var_unchecked %.4f, var_Precision %.4f, var_Recall %.4f, var_F1-score %.4f, var_AUC %.4f' % (np.array(history['loss']).var(), np.array(history['accuracy']).var(), np.array(history['arithmetic']).var(), np.array(history['reentrancy']).var(), np.array(history['timestamp']).var(), np.array(history['unchecked']).var(), np.array(history['precision']).var(), np.array(history['recall']).var(), np.array(history['f1']).var(), np.array(history['auc']).var()), file=f)
        history_all['accuracy'].append(np.array(history['accuracy']).mean())
        history_all['loss'].append(np.array(history['loss']).mean())
        history_all['auc'].append(np.array(history['auc']).mean())
        history_all['precision'].append(np.array(history['precision']).mean())
        history_all['recall'].append(np.array(history['recall']).mean())
        history_all['f1'].append(np.array(history['f1']).mean())
        history_all['arithmetic'].append(np.array(history['arithmetic']).mean())
        history_all['reentrancy'].append(np.array(history['reentrancy']).mean())
        history_all['timestamp'].append(np.array(history['timestamp']).mean())
        history_all['unchecked'].append(np.array(history['unchecked']).mean())
    with open('./results/'+'finetune_9fold'+d_type+'.txt','a') as f:
        print('ave_Loss %.4f, ave_Acc %.4f, ave_arithmetic %.4f, ave_reentrancy %.4f, ave_timestamp %.4f, ave_unchecked %.4f, ave_Precision %.4f, ave_Recall %.4f, ave_F1-score %.4f, ave_AUC %.4f' % (np.array(history_all['loss']).mean(), np.array(history_all['accuracy']).mean(), np.array(history_all['arithmetic']).mean(), np.array(history_all['reentrancy']).mean(), np.array(history_all['timestamp']).mean(), np.array(history_all['unchecked']).mean(), np.array(history_all['precision']).mean(), np.array(history_all['recall']).mean(), np.array(history_all['f1']).mean(), np.array(history_all['auc']).mean()), file=f)
        print('var_Loss %.4f, var_Acc %.4f, var_arithmetic %.4f, var_reentrancy %.4f, var_timestamp %.4f, var_unchecked %.4f, var_Precision %.4f, var_Recall %.4f, var_F1-score %.4f, var_AUC %.4f' % (np.array(history_all['loss']).var(), np.array(history_all['accuracy']).var(), np.array(history_all['arithmetic']).var(), np.array(history_all['reentrancy']).var(), np.array(history_all['timestamp']).var(), np.array(history_all['unchecked']).var(), np.array(history_all['precision']).var(), np.array(history_all['recall']).var(), np.array(history_all['f1']).var(), np.array(history_all['auc']).var()), file=f)

/home/bmw_lab/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


train_loss:  0.5701241952000242
Validation loss decreased (inf --> 0.482536).  Saving model ...


/home/bmw_lab/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


train_loss:  0.5131464950966113
Validation loss decreased (0.482536 --> 0.429838).  Saving model ...


/home/bmw_lab/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


...

train_loss:  0.23336935653743973
Validation loss decreased (0.302439 --> 0.293404).  Saving model ...


/home/bmw_lab/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


train_loss:  0.2298560212085764
EarlyStopping counter: 1 out of 30


/home/bmw_lab/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


KeyboardInterrupt: 

In [ ]:
# d_type = 'arithmetic'
# detect_types = ['arithmetic','reentrancy','timestamp','unchecked']
# detect_types = ['multi-label']

# for item in detect_types:
#     history = {'loss':[], 'auc':[], 'precision':[], 'recall':[], 'f1':[], 'accuracy':[], 'arithmetic':[], 'reentrancy':[], 'timestamp':[], 'unchecked':[]}
#     d_type = item
#     print(d_type,' start training!')
#     for i in tqdm(range(30)):
#         start_training(i)
#     with open('./simgrace/'+'finetune_'+d_type+'.txt','a') as f:
#         print('ave_Loss %.4f, ave_Acc %.4f, ave_arithmetic %.4f, ave_reentrancy %.4f, ave_timestamp %.4f, ave_unchecked %.4f, ave_Precision %.4f, ave_Recall %.4f, ave_F1-score %.4f, ave_AUC %.4f' % (np.array(history['loss']).mean(), np.array(history['accuracy']).mean(), np.array(history['arithmetic']).mean(), np.array(history['reentrancy']).mean(), np.array(history['timestamp']).mean(), np.array(history['unchecked']).mean(), np.array(history['precision']).mean(), np.array(history['recall']).mean(), np.array(history['f1']).mean(), np.array(history['auc']).mean()), file=f)
#         print('var_Loss %.4f, var_Acc %.4f, var_arithmetic %.4f, var_reentrancy %.4f, var_timestamp %.4f, var_unchecked %.4f, var_Precision %.4f, var_Recall %.4f, var_F1-score %.4f, var_AUC %.4f' % (np.array(history['loss']).var(), np.array(history['accuracy']).var(), np.array(history['arithmetic']).var(), np.array(history['reentrancy']).var(), np.array(history['timestamp']).var(), np.array(history['unchecked']).var(), np.array(history['precision']).var(), np.array(history['recall']).var(), np.array(history['f1']).var(), np.array(history['auc']).var()), file=f)

In [ ]:
# print('Loss %.4f' % 0.123324)

In [ ]:
# for i in np.linspace(0,1,100):
#     loss, precision, recall, F1, accuracy, auc = test(i)
#     print('thresh: ',i, 'Loss {}, Acc {}, Precision {}, recall {}, F1-score {}, AUC {}'.format(loss, accuracy, precision, recall, F1, auc))